# 100 - Independant NER layers - Datasets generation

Outputs:
Train, dev & test datasets for multilayers NER experiment 1 (with ref dataset) and experiment 2 (with pero OCR dataset)

<b>Experiment 1 : Groundtruth dataset</b>
* `m1-experiment_1_prepared_dataset_ref_io_camembert_ner`
* `m1-experiment_1_prepared_dataset_ref_io_pretrained_camembert_ner`
* `m1-experiment_1_prepared_dataset_ref_iob2_camembert_ner`
* `m1-experiment_1_prepared_dataset_ref_iob2_pretrained_camembert_ner`

<b>Experiment 2 : Pero OCR dataset</b>
* `m1-experiment_2_prepared_dataset_pero_ocr_io_camembert_ner`
* `m1-experiment_2_prepared_dataset_pero_ocr_io_pretrained_camembert_ner`
* `m1-experiment_2_prepared_dataset_pero_ocr_iob2_camembert_ner`
* `m1-experiment_2_prepared_dataset_pero_ocr_iob2_pretrained_camembert_ner`

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

2


## Initialization

In [2]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR').resolve()
  OUT_BASE = Path('/work/stual/res_ICDAR/method_1').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m1_independant_ner_layers', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m1_independant_ner_layers
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_1


## Constants

**Tokenizer load in *util_io.py* and *util_iob2.py* has to be the same that the model to fine-tune.<br>
Please check *_convert_tokenizer* var in those files and adapt MODEL_NAME variable in this notebook before creating datasets.** 

In [3]:
# Save on disk
from multihead_utils.util_io import _convert_tokenizer
print("Tokenizer called in util_io.py")
_convert_tokenizer.name_or_path

2023-01-19 16:35:58.694315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 16:36:00.252013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-19 16:36:00.252138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-19 16:36:00.252152: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Tokenizer called in util_io.py


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'HueyNemud/das22-10-camembert_pretrained'

In [4]:
from multihead_utils.util_iob2 import _convert_tokenizer
print("Tokenizer called in util_iob2.py")
_convert_tokenizer.name_or_path

Tokenizer called in util_iob2.py


[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'HueyNemud/das22-10-camembert_pretrained'

In [5]:
# GLOBAL CONSTANTS
import config
import os
from pathlib import Path

config.SPLIT_SEED = 42 # Random seed used in train/dev/test. Do not change it if you want to recreate the paper results.
config.DEBUG = False # If true, text versions of the spacy datasets will be saved along with the .spacy files.

MODEL_NAME = "pretrained_camembert_ner" #Used for folder name, no space, different from real model name
#camembert_ner OR
#pretrained_camembert_ner

# 01. Experiment #1 : Reference dataset

In [6]:
import numpy as np
import pandas as pd
import csv

GOLD_REF = DATASETS / "41-ner_ref_from_pero/gold.csv"
assert GOLD_REF.exists()

with open(GOLD_REF,'r',encoding='utf8') as f:
    lines = f.readlines()
    res = []
    for line in lines:
        l = line.split('", "')
        res.append([l[0][1:],l[1][:-2]])
gold_reference = pd.DataFrame(res,columns=["ner_xml","book"])

In [7]:
#TITRE-H and TITRE-P labels to transformers NER Pipeline
for i in range(len(gold_reference)):
    if '<TITRE-H>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-H','TITREH')
    if '<TITRE-P>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-P','TITREP')

gold_reference

,ner_xml,book
0,"<PER>Dufant (Victor)</PER>, <ACT>libraire</ACT...",Bottin1_1820
1,"<PER>Dufay</PER>, <ACT>essayeur du commerce</A...",Bottin1_1820
2,"<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SP...",Bottin1_1820
3,"<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...",Bottin1_1820
4,"<PER>Dufeu</PER>, <ACT>charcutier</ACT>, <SPAT...",Bottin1_1820
...,...,...
8440,"<PER>Lamarche</PER>, <ACT>géographe</ACT> , <S...",Notables_communaux_seine_1801
8441,"<PER>Lamarck</PER>, <ACT>membre de l'institut<...",Notables_communaux_seine_1801
8442,"<PER>Lamare</PER>, <ACT>notaire</ACT>, <SPAT><...",Notables_communaux_seine_1801
8443,"<PER>Lamarre</PER> , <ACT>carrier</ACT>, <SPAT...",Notables_communaux_seine_1801


In [9]:
from multihead_utils.multihead_dataset_util import train_dev_test_split, unwrap # Local imports
from sklearn.model_selection import train_test_split

# CONSTANTS
MIN_TRAINSET_SIZE = 30

 # Split 72/8/20% w. stratified sampling on directories names
train, dev, test = train_dev_test_split(gold_reference.to_numpy())
print("Dev : "+str(len(dev)))
print("Test : "+str(len(test)))

# Iteratively split the trainset in half to create smaller trainsets
exp1_trainsets = [train]
t_len = len(train)

while True:
    try:
        current = exp1_trainsets[-1]
        _, groups = unwrap(current)
        smaller, rest = train_test_split(
            current,
            train_size=0.5,
            shuffle=True,
            random_state=config.SPLIT_SEED,
            stratify=groups,
        )
        t_len = len(rest)
        if t_len < MIN_TRAINSET_SIZE:
            break
        exp1_trainsets.append(smaller)

    except ValueError:
        # Stop now if we encounter the error "The least populated class in y has only 1 member".
        break

[len(s) for s in exp1_trainsets] # Should be 

Dev : 676
Test : 1685


[6084, 3042, 1521, 760, 380, 190, 95, 47]

In [10]:
# Sanity checks

# Dev set should contain 676 examples
assert len(dev) == 676

# Test set should contain 1685 examples
assert len(test) == 1685

# Lenghts of exp1_trainsets should be fixed
assert sorted([len(s) for s in exp1_trainsets] ) == sorted([6084, 3042, 1521, 760, 380, 190, 95, 47])

In [11]:
from multihead_utils.tools import createStatsTab

createStatsTab(train,dev,test)

,Train,Dev,Test,All
Entity type,,,,
PER,6085,676,1685,8446
ACT,4568,519,1094,6181
DESC,262,30,79,371
TITRE,11,2,0,13
TITREH,225,33,43,301
TITREP,54,7,33,94
SPAT,6214,694,1747,8655
LOC,6871,762,1788,9421
FT,55,7,14,76


### IO Labels

In [12]:
# Save on disk
from multihead_utils.util_io import save_dataset # Local import

output_directory = OUT_BASE / f"m1-experiment_1_prepared_dataset_ref_io_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
   
for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset(output_directory, datasets, ["train","dev","test"], suffix=len(train))
output_directory

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

PosixPath('/work/stual/res_ICDAR/method_1/m1-experiment_1_prepared_dataset_ref_io_pretrained_camembert_ner')

### IOB2 Labels

In [13]:
# Save on disk
from multihead_utils.util_iob2 import save_dataset # Local import

output_directory = OUT_BASE / f"m1-experiment_1_prepared_dataset_ref_iob2_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
   
for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset(output_directory, datasets, ["train","dev","test"], suffix=len(train))
output_directory

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

PosixPath('/work/stual/res_ICDAR/method_1/m1-experiment_1_prepared_dataset_ref_iob2_pretrained_camembert_ner')

## 02. Experiment 2 : Pero OCR dataset

In [14]:
import numpy as np
import pandas as pd
import csv

GOLD_REF = DATASETS / "31-ner_align_pero/gold.csv"

gold_reference = pd.read_csv(GOLD_REF, header=None, names=["ner_xml","book"],skipinitialspace='True')

In [15]:
#TITRE-H and TITRE-P labels to transformers NER Pipeline
for i in range(len(gold_reference)):
    if '<TITRE-H>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-H','TITREH')
    if '<TITRE-P>' in gold_reference['ner_xml'][i]:
        gold_reference['ner_xml'][i] = gold_reference['ner_xml'][i].replace('TITRE-P','TITREP')
gold_reference

,ner_xml,book
0,"☞ T <PER>Dufant (Victor)</PER>, <ACT>libraire...",Bottin1_1820
1,"<PER>Dutay</PER>, <ACT>essayeur du commerce</A...",Bottin1_1820
2,"<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SP...",Bottin1_1820
3,"<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...",Bottin1_1820
4,"Y ☞ <PER>Dnten</PER>,<ACT>charentier</ACT>, <S...",Bottin1_1820
...,...,...
8440,"<PER>Lamarche</PER>, <ACT>geographe</ACT> , <S...",Notables_communaux_seine_1801
8441,"<PER>Lamarck</PER>, <ACT>membre de l&apos;inst...",Notables_communaux_seine_1801
8442,"<PER>Lamare</PER> , <ACT>notaire</ACT>, <SPAT>...",Notables_communaux_seine_1801
8443,"<PER>Lamarre</PER>, <ACT>carrier</ACT>, <SPAT>...",Notables_communaux_seine_1801


In [17]:
from multihead_utils.multihead_dataset_util import train_dev_test_split, unwrap # Local imports
from sklearn.model_selection import train_test_split

# CONSTANTS
MIN_TRAINSET_SIZE = 30

 # Split 72/8/20% w. stratified sampling on directories names
train, dev, test = train_dev_test_split(gold_reference.to_numpy())
print("Dev : "+str(len(dev)))
print("Test : "+str(len(test)))

# Iteratively split the trainset in half to create smaller trainsets
exp1_trainsets = [train]
t_len = len(train)

while True:
    try:
        current = exp1_trainsets[-1]
        _, groups = unwrap(current)
        smaller, rest = train_test_split(
            current,
            train_size=0.5,
            shuffle=True,
            random_state=config.SPLIT_SEED,
            stratify=groups,
        )
        t_len = len(rest)
        if t_len < MIN_TRAINSET_SIZE:
            break
        exp1_trainsets.append(smaller)

    except ValueError:
        # Stop now if we encounter the error "The least populated class in y has only 1 member".
        break

[len(s) for s in exp1_trainsets] # Should be 

Dev : 676
Test : 1685


[6084, 3042, 1521, 760, 380, 190, 95, 47]

In [18]:
# Sanity checks

# Dev set should contain 676 examples
assert len(dev) == 676

# Test set should contain 1685 examples
assert len(test) == 1685

# Lenghts of exp1_trainsets should be fixed
assert sorted([len(s) for s in exp1_trainsets] ) == sorted([6084, 3042, 1521, 760, 380, 190, 95, 47])

In [19]:
from multihead_utils.tools import createStatsTab

createStatsTab(train,dev,test)

,Train,Dev,Test,All
Entity type,,,,
PER,6085,676,1685,8446
ACT,4568,519,1094,6181
DESC,262,30,79,371
TITRE,11,2,0,13
TITREH,225,33,43,301
TITREP,54,7,33,94
SPAT,6214,694,1747,8655
LOC,6871,762,1788,9421
FT,55,7,14,76


### IO Labels

In [20]:
# Save on disk
from multihead_utils.util_io import save_dataset # Local import

output_directory = OUT_BASE / f"m1-experiment_2_prepared_dataset_pero_ocr_io_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
   
for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset(output_directory, datasets, ["train","dev","test"], suffix=len(train))
output_directory

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

PosixPath('/work/stual/res_ICDAR/method_1/m1-experiment_2_prepared_dataset_pero_ocr_io_pretrained_camembert_ner')

In [21]:
#Example
import pandas as pd
from datasets import load_from_disk

dataset_root = f"huggingface_6084"
datasetdir = output_directory / dataset_root
train_dev_test = load_from_disk(datasetdir)

print(train_dev_test["train"][3]["tokens"])
print(train_dev_test["train"][3]["ner_tags_niv1"])
print(train_dev_test["train"][3]["ner_tags_niv2"])

['Lemarchand', ',', 'dépôt', 'de', 'velours', "d'Utrecht", 'et', 'moquettes', ',', 'r.', 'du', 'Pet.-Carreau', ',', '3', '.']
['I-PER', 'O', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'I-ACT', 'O', 'I-SPAT', 'I-SPAT', 'I-SPAT', 'I-SPAT', 'I-SPAT', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'I-CARDINAL', 'O']


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### IOB2 Labels

In [22]:
# Save on disk
from multihead_utils.util_iob2 import save_dataset # Local import

output_directory = OUT_BASE / f"m1-experiment_2_prepared_dataset_pero_ocr_iob2_{MODEL_NAME}"
output_directory.mkdir(exist_ok=True, parents=True) # Create if necessary
   
for train in exp1_trainsets:
    datasets = [train, dev, test]
    save_dataset(output_directory, datasets, ["train","dev","test"], suffix=len(train))
output_directory

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

PosixPath('/work/stual/res_ICDAR/method_1/m1-experiment_2_prepared_dataset_pero_ocr_iob2_pretrained_camembert_ner')